https://terryoy.github.io/2014/03/using-ctypes-to-wrap-c-library.html

https://stackoverflow.com/questions/5081875/ctypes-beginner

In [1]:
import ctypes
import sys

# Import DLLs

In [2]:
std_lib = ctypes.WinDLL('C:\Windows\System32\msvcrt.dll')

# ctypes

### int

In [74]:
my_int = ctypes.c_int(20)

In [75]:
my_int

c_long(20)

In [76]:
my_int.value

20

In [77]:
my_int.value = 10

In [78]:
my_int

c_long(10)

In [15]:
ctypes.c_int(1.2)

TypeError: int expected instead of float

In [81]:
ctypes.c_uint32(1)

c_ulong(1)

In [84]:
ctypes.c_uint32(-1)

c_ulong(4294967295)

In [82]:
-1 + 2**32

4294967295

In [86]:
ctypes.c_uint32('a')

TypeError: an integer is required (got type bytes)

### Char

In [87]:
'a'

'a'

In [88]:
ctypes.c_char('a')

TypeError: one character bytes, bytearray or integer expected

In [96]:
sys.getsizeof('a')

50

In [97]:
sys.getsizeof('a'.encode('ascii'))

34

In [100]:
ctypes.c_char('a'.encode('ascii'))

c_char(b'a')

### String

In C, string is an array of char, terminated by `"\0"`.

That is why a variable, containing a string, always has `char*` type.

That is why, when passing a Python string as argument, you need to wrap it with `ctypes.c_char_p()`. It's also necessary to convert it to binary format `b'Python_string'`

In [93]:
my_str = ctypes.c_char_p(b'abc')

In [94]:
my_str

c_char_p(2024871322600)

In [95]:
my_str.value

b'abc'

#### atoi

`int atoi (const char * str);`

In [185]:
std_lib.atoi.restype = ctypes.c_int

In [13]:
'123'

'123'

In [14]:
'123'.encode('ascii')

b'123'

In [24]:
my_str = ctypes.c_char_p('-123'.encode('ascii'))

In [20]:
my_str

c_char_p(2245264458048)

In [21]:
my_str.value

b'-123.5'

In [23]:
std_lib.atoi(my_str)

-123

##### Test: by reference and c_char_p

In [188]:
std_lib.atoi(ctypes.c_char_p(b'123'))

123

In [189]:
std_lib.atoi(ctypes.byref(b'123'))

TypeError: byref() argument must be a ctypes instance, not 'bytes'

##### Test ctypes.POINTER

In [190]:
my_c_char_p = ctypes.POINTER(ctypes.c_char)

In [191]:
my_c_char_p(b'123')

TypeError: expected c_char instead of bytes

`ctypes.POINTER(ctypes.c_char)` is not `ctypes.c_char_p`

In [192]:
ctypes.c_char_p(b'123')

c_char_p(2024871324280)

#### atof

`double atof (const char* str);`

In [3]:
my_str = ctypes.c_char_p('-123.5'.encode('ascii'))

In [4]:
my_str

c_char_p(2024870649640)

In [5]:
my_str.value

b'-123.5'

It is important to specify correct restype to avoid incorrect C->Python translation of the result

In [7]:
std_lib.atof(my_str)

1870334016

In [8]:
std_lib.atof.restype = ctypes.c_float

In [9]:
std_lib.atof(my_str)

0.0

In [66]:
std_lib.atof.restype = ctypes.c_double  # Correct

In [11]:
std_lib.atof(my_str)

-123.5

```None, integers, bytes objects and (unicode) strings are the only native Python objects that can directly be used as parameters in these function calls. None is passed as a C NULL pointer, bytes objects and strings are passed as pointer to the memory block that contains their data (char * or wchar_t *). Python integers are passed as the platforms default C int type, their value is masked to fit into the C type.```

In [69]:
std_lib.atof(b'12.3')

12.3

#### ctypes.create_string_buffer()

In [42]:
str_buf = ctypes.create_string_buffer(b'abc', size=10)

In [43]:
str_buf

In [49]:
ctypes.sizeof(str_buf)

10

In [48]:
sys.getsizeof(str_buf)

128

In [44]:
str_buf.value

b'abc'

In [45]:
str_buf.raw

b'abc\x00\x00\x00\x00\x00\x00\x00'

In [46]:
str_buf.value = b'defg'

In [47]:
str_buf.raw

b'defg\x00\x00\x00\x00\x00\x00'

To test using mutable char arrays, example with spriintf function

`int sprintf(char *buffer_for_output, const char *str_to_print,...);`

Create `char*` `str_to_print`

In [59]:
str_to_print = ctypes.c_char_p(b'ABC123')

In [60]:
str_to_print.value

b'ABC123'

In [61]:
std_lib.sprintf(str_buf, str_to_print)

6

In [62]:
str_buf.raw

b'ABC123\x00\x00\x00\x00'

#### Alternative way to create (immutable) char array [don't know where it might be used]

In [25]:
buffer_for_output = ctypes.c_char * 10

In [26]:
buffer_for_output

__main__.c_char_Array_10

In [27]:
ctypes.sizeof(buffer_for_output)

10

In [28]:
buffer_for_output.value

<attribute 'value' of 'c_char_Array_10' objects>

Pass to `sprintf()` gives error

In [63]:
std_lib.sprintf(buffer_for_output, str_to_print)

ArgumentError: argument 1: <class 'TypeError'>: Don't know how to convert parameter 1

### ctypes. byref(), pointer(), POINTER()

`byref(var_name)` -- use to pass a poiner to var_name object as function argument (without creating actual poiner instance)

`pointer(var_name)` -- create pointer object, pointing at var_name object

`c_come_type_p = POINTER(c_come_type)` -- define new type (create an new class, a pointer to object of type `c_some_type`)

In [ ]:
a = ctypes.pointer(b) = ctypes.POINTER(ctypes.c_int)

In [173]:
a = 2

In [174]:
int_point = ctypes.POINTER(ctypes.c_int32)

In [175]:
b = int_point(ctypes.c_int32(a)) 

In [176]:
b.contents

c_long(2)

#### ctypes.byref()

```Sometimes a C api function expects a pointer to a data type as parameter, probably to write into the corresponding location, or if the data is too large to be passed by value. This is also known as passing parameters by reference.```

```ctypes exports the byref() function which is used to pass parameters by reference. The same effect can be achieved with the pointer() function, although pointer() does a lot more work since it constructs a real pointer object, so it is faster to use byref() if you don’t need the pointer object in Python itself```

`int sscanf(const char *str, const char *format, ...)`

str − This is the C string that the function processes as its source to retrieve the data.

format − This is the C string that contains one or more of the following items: Whitespace character, Non-whitespace character and Format specifiers

A format specifier follows this prototype: [=%[*][width][modifiers]type=]

In [100]:
s = ctypes.create_string_buffer(10)
a = ctypes.c_int32()
b = ctypes.c_int32()

std_lib.sscanf(
    ctypes.c_char_p(b'abc 1 2'),
    ctypes.c_char_p(b'%s %d %d'),
    s, ctypes.byref(a), ctypes.byref(b)
)

3

In [101]:
a

c_long(1)

In [102]:
b

c_long(2)

In [104]:
s.value

b'abc'

#### ctypes.pointer()

```ctypes exports the byref() function which is used to pass parameters by reference. The same effect can be achieved with the pointer() function, although pointer() does a lot more work since it constructs a real pointer object, so it is faster to use byref() if you don’t need the pointer object in Python itself```

In [150]:
a = ctypes.c_int32(5)

In [151]:
a

c_long(5)

In [152]:
ptr= ctypes.pointer(a)

In [153]:
ptr

```Pointer instances have a contents attribute which returns an object equivalent to the object to which the pointer points. Note that ctypes does not have OOR (original object return), it constructs a new, equivalent object each time you retrieve an attribute
```

In [154]:
ptr.contents

c_long(5)

In [155]:
ptr.contents == a

False

In [117]:
ptr.contents.value

5

In [118]:
ptr.contents.value = 6

In [119]:
ptr.contents.value

6

#### ctypes.POINTER()

`PONITER(some_ctypes_type)` creates a new type (new class) - pointer to `some_ctypes_type`

In [157]:
new_ptr_class = ctypes.POINTER(ctypes.c_int32)

In [158]:
new_ptr_class

ctypes.wintypes.LP_c_long

In [160]:
a = ctypes.c_int32(12)

In [161]:
ptr_to_a = new_ptr_class(a)

In [163]:
ptr_to_a

In [165]:
ptr_to_a.contents.value

12

### ctypes.sizeof()

`ctypes.sizeof()` - physical memory size of C data type

`sys.getsizeof()` - physical memory size of a Python object

In [69]:
help(ctypes.sizeof)

Help on built-in function sizeof in module _ctypes:

sizeof(...)
    sizeof(C type) -> integer
    sizeof(C instance) -> integer
    Return the size in bytes of a C instance



In [71]:
help(sys.getsizeof)

Help on built-in function getsizeof in module sys:

getsizeof(...)
    getsizeof(object, default) -> int
    
    Return the size of object in bytes.



In [31]:
ctypes.sizeof(ctypes.c_int)  # C int

4

In [32]:
ctypes.sizeof(ctypes.c_int(5))  # C int

4

In [33]:
sys.getsizeof(5)  # Python int object

28

sizeof different C data types on this machine

Boolean

In [37]:
ctypes.sizeof(ctypes.c_bool)

1

In [38]:
sys.getsizeof(True)

28

Character / pointer to char

In [39]:
ctypes.sizeof(ctypes.c_char)

1

In [40]:
ctypes.sizeof(ctypes.c_char_p)

8

Integer

In [34]:
ctypes.sizeof(ctypes.c_int)

4

In [35]:
ctypes.sizeof(ctypes.c_int16)

2

In [36]:
ctypes.sizeof(ctypes.c_int32)

4

In [10]:
ctypes.sizeof(ctypes.c_int64)

8

In [12]:
ctypes.sizeof(ctypes.c_long)

4

In [13]:
ctypes.sizeof(ctypes.c_longlong)

8

Float / Double

In [41]:
ctypes.sizeof(ctypes.c_float)

4

In [42]:
ctypes.sizeof(ctypes.c_double)

8

In [14]:
ctypes.sizeof(ctypes.c_longdouble)

8

In [53]:
sys.getsizeof(10.23e-30)

24

### Limits and Overflows

In [57]:
2**31 - 1

2147483647

In [62]:
ctypes.c_int32(2147483647)

c_long(2147483647)

In [68]:
ctypes.c_int32(2147483647 + 1)

c_long(-2147483648)

## Functions

In [3]:
std_lib.abs(-10.1)

ArgumentError: argument 1: <class 'TypeError'>: Don't know how to convert parameter 1

In [5]:
std_lib.abs(ctypes.c_float(-10.1))

1054762598

In [6]:
ctypes.c_float(std_lib.abs(ctypes.c_float(-10.1)))

c_float(1054762624.0)

#### argtypes

In [6]:
std_lib.abs.argtypes = [ctypes.c_float]

In [7]:
std_lib.abs(-10.1)

1054762598

#### restype

In [8]:
std_lib.abs.restype = ctypes.c_float

In [9]:
std_lib.abs(-10.1)

-10.100000381469727

In [ ]:
int_var = ctypes.c_int(15)

In [ ]:
int_var

In [ ]:
int_var.value

##### byref()

Can only be used in function calls

In [ ]:
ctypes.byref(int_val)

##### pointer()

In [ ]:
int_val_ptr = ctypes.pointer(int_val)

In [ ]:
int_val_ptr

In [ ]:
int_val_ptr.contents

In [ ]:
int_val_ptr.contents.value

##### POINTER()

### Simple functions

In [ ]:
win_dll.rand()

In [ ]:
win_dll.atoi("345")

In [31]:
std_lib.atoi(ctypes.c_char_p("345".encode('ascii')))

345

In [ ]:
win_dll.atoi.argtypes = [ctypes.c_char_p]
win_dll.atoi.restype = ctypes.c_int

In [ ]:
win_dll.atoi("12")

In [ ]:
win_dll.atoi(ctypes.byref(ctypes.c_char("300")))